# Analyzing eBay Car Sales
In this project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle. We've made a few modifications from the original dataset that was uploaded to Kaggle:

* We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
* We dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

The aim of this project is to clean the data and analyze the included used car listings.

In [1]:
import pandas as pd
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [2]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


We can make the following observations:

* The dataset contains 20 columns, most of which are strings.
* Some columns have null values, but none have more than ~20% null values.
* The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [3]:
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [4]:
autos.rename(columns = {
                        'dateCrawled': 'date_crawled',
                        'offerType': 'offer_type',
                        'vehicleType': 'vehicle_type',
                        'yearOfRegistration': 'registration_year',
                        'powerPS': 'power_ps',
                        'monthOfRegistration': 'registration_month',
                        'fuelType': 'fuel_type',
                        'notRepairedDamage': 'unrepaired_damage',
                        'dateCreated': 'ad_created',
                        'nrOfPictures': 'num_pictures',
                        'postalCode': 'postal_code',
                        'lastSeen': 'last_seen'
                        }
            , inplace=True)

Now let's do some basic data exploration to determine what other cleaning tasks need to be done.

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 17:37:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The `seller`, `offer_type`, and `num_pictures` columns can be dropped, because no useful information can be extracted.

In [6]:
autos.drop(['seller', 'offer_type', 'num_pictures'], axis=1, inplace=True)

We can also see that the `price` and `odometer` columns are numeric values stored as text. We'll remove non-numeric characters and then convert the columns to a numeric dtype.

In [7]:
autos['odometer'] = (autos['odometer']
                     .str.replace(',', '')
                     .str.strip('km')
                     .astype('int')
                    )

In [8]:
autos['price'] = (autos['price']
                  .str.replace(',', '')
                  .str.strip('$')
                  .astype('float')
                 )
autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)

Let's continue exploring the data, specifically looking for data that doesn't look right. We'll start by analyzing the `odometer_km` and `price` columns.

In [9]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [10]:
autos['odometer_km'].unique()

array([150000,  70000,  50000,  80000,  10000,  30000, 125000,  90000,
        20000,  60000,   5000, 100000,  40000])

In [11]:
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

The `odometer_km` column has 50,000 non-null entries, ranging from 5,000 km to 150,000 km, with the majority of them being 150K. There does not seem to be any outliers here.

In [12]:
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [13]:
autos['price'].unique().shape

(2357,)

In [14]:
autos['price'].value_counts().sort_index(ascending=False).head(15)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
Name: price, dtype: int64

In [15]:
autos['price'].value_counts().sort_index(ascending=True).head(15)

0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
Name: price, dtype: int64

While there are a lot of unique values for the `price`, a price of `0.0` would not make any sense, and likewise the amount of cars that have a value of `1.0` also skews the data. On the other end of the spectrum, there is a big leap in the value of prices after `350000.0`. We can omit these extreme values from our dataset.

In [16]:
autos = autos.loc[(autos['price'] > 1.0) & (autos['price'] <= 350000.0), :]

Let's now move on to the date columns and understand the date range the data covers.

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

* `date_crawled`: added by the crawler
* `last_seen`: added by the crawler
* `ad_created`: from the website
* `registration_month`: from the website
* `registration_year`: from the website

Right now, the `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively.

In [17]:
autos[['date_crawled','ad_created','last_seen']].head()

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


We see that the first 10 characters represent the day, followed by a timestamp. To understand the date range, we can extract just the date values, use `Series.value_counts()` to generate a distribution, and then sort by the index.

In [18]:
autos['date_crawled'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.025367
2016-03-06    0.014068
2016-03-07    0.036047
2016-03-08    0.033279
2016-03-09    0.033052
2016-03-10    0.032205
2016-03-11    0.032597
2016-03-12    0.036956
2016-03-13    0.015658
2016-03-14    0.036625
2016-03-15    0.034270
2016-03-16    0.029519
2016-03-17    0.031626
2016-03-18    0.012911
2016-03-19    0.034746
2016-03-20    0.037803
2016-03-21    0.037307
2016-03-22    0.032928
2016-03-23    0.032267
2016-03-24    0.029395
2016-03-25    0.031564
2016-03-26    0.032246
2016-03-27    0.031131
2016-03-28    0.034849
2016-03-29    0.034126
2016-03-30    0.033733
2016-03-31    0.031812
2016-04-01    0.033733
2016-04-02    0.035489
2016-04-03    0.038588
2016-04-04    0.036481
2016-04-05    0.013076
2016-04-06    0.003161
2016-04-07    0.001384
Name: date_crawled, dtype: float64

We can see that the crawler started running in the beginning of March, and ran consistently until the first week of April.

In [19]:
autos['ad_created'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033114
2016-03-10    0.031916
2016-03-11    0.032928
2016-03-12    0.036770
2016-03-13    0.017022
2016-03-14    0.035262
2016-03-15    0.034002
2016-03-16    0.030036
2016-03-17    0.031275
2016-03-18    0.013593
2016-03-19    0.033630
2016-03-20    0.037865
2016-03-21 

Most ads from the dataset were created during the one month duration of the crawler, and some ads created a couple of months before the crawler started. There are a handful of ads that were created in mid 2015.

In [20]:
autos['last_seen'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.001074
2016-03-06    0.004338
2016-03-07    0.005412
2016-03-08    0.007375
2016-03-09    0.009626
2016-03-10    0.010618
2016-03-11    0.012374
2016-03-12    0.023797
2016-03-13    0.008862
2016-03-14    0.012622
2016-03-15    0.015865
2016-03-16    0.016443
2016-03-17    0.028073
2016-03-18    0.007333
2016-03-19    0.015824
2016-03-20    0.020637
2016-03-21    0.020616
2016-03-22    0.021380
2016-03-23    0.018592
2016-03-24    0.019748
2016-03-25    0.019191
2016-03-26    0.016815
2016-03-27    0.015617
2016-03-28    0.020885
2016-03-29    0.022331
2016-03-30    0.024747
2016-03-31    0.023839
2016-04-01    0.022868
2016-04-02    0.024851
2016-04-03    0.025202
2016-04-04    0.024500
2016-04-05    0.124935
2016-04-06    0.221591
2016-04-07    0.132021
Name: last_seen, dtype: float64

The `last_seen` column date range coincides with the date range of the crawler, as expected. The majority of the `last_seen` occur during the last few days of the crawler, also as expected.

In [21]:
autos['registration_year'].describe()

count    48409.000000
mean      2004.774319
std         88.783278
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

One thing that stands out from the distribution of the `registration_year` column is that it contains some odd values:

* The minimum value is 1000, before cars were invented
* The maximum value is 9999, many years into the future

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [22]:
autos['registration_year'].value_counts().sort_index(ascending=True).head(15)

1000    1
1001    1
1111    1
1800    2
1910    4
1927    1
1929    1
1931    1
1934    2
1937    4
1938    1
1939    1
1941    2
1943    1
1948    1
Name: registration_year, dtype: int64

In [23]:
autos.loc[(autos['registration_year']>=1910) & (autos['registration_year']<=1948), :]

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
2221,2016-03-15 14:57:07,Sehr_seltener_Oldtimer_Opel_1210_zum_Restaurieren,3350.0,control,andere,1934,manuell,0,andere,5000,0,benzin,opel,ja,2016-03-15 00:00:00,49828,2016-04-06 06:17:51
2573,2016-03-19 22:51:25,Hanomag_rekord_15k_Suche_ersatz_teile,3000.0,test,andere,1934,NaN,0,NaN,90000,1,benzin,sonstige_autos,nein,2016-03-19 00:00:00,90489,2016-03-19 22:51:25
11047,2016-03-08 20:50:10,Andere_Simca_5_Fourgonette_Kombilimousine,17500.0,control,kombi,1948,manuell,0,NaN,60000,6,benzin,sonstige_autos,nein,2016-03-08 00:00:00,47546,2016-04-05 21:15:42
11246,2016-03-26 19:49:59,Ford_Model_A_Roadster_Deluxe_1931,27500.0,control,cabrio,1931,manuell,39,andere,10000,7,benzin,ford,nein,2016-03-26 00:00:00,9322,2016-04-06 09:46:59
11585,2016-03-11 21:48:36,Volkswagen__VW_Typ_82,41900.0,test,cabrio,1943,NaN,0,andere,100000,7,NaN,volkswagen,ja,2016-03-11 00:00:00,84174,2016-03-21 13:18:05
13963,2016-03-20 17:51:49,Mercedes_Benz_L1500S_Wehrmacht_/_Luftwaffe___F...,26900.0,test,andere,1941,manuell,60,andere,60000,7,benzin,mercedes_benz,nein,2016-03-20 00:00:00,38723,2016-04-07 01:17:51
21416,2016-03-12 08:36:21,Essex_super_six__Ford_A,16500.0,control,cabrio,1927,manuell,40,andere,5000,5,benzin,ford,NaN,2016-03-12 00:00:00,74821,2016-03-15 12:45:12
21421,2016-03-05 17:45:32,Ford_Business_Coupe_Hotrod_Projekt.1937,7000.0,test,coupe,1937,manuell,85,andere,5000,8,benzin,ford,ja,2016-03-05 00:00:00,8359,2016-04-07 10:44:39
22101,2016-03-09 16:51:17,BMW_Andere,11500.0,test,cabrio,1929,manuell,15,andere,5000,1,NaN,bmw,ja,2016-03-09 00:00:00,70569,2016-04-07 06:17:11
22659,2016-03-14 08:51:18,Opel_Corsa_B,500.0,test,NaN,1910,NaN,0,corsa,150000,0,NaN,opel,NaN,2016-03-14 00:00:00,52393,2016-04-03 07:53:55


None of these results for the year 1910 make any sense:

* The two Opel models both started production in the 1980's
* The Renault Twingo strated production in the 1990's
* Motorhaube translates to 'hood' in German, and judging by the price that's probably what's for sale (not an actual automobile)

All results for 1910 can be omitted.

The other results after 1910 are plausible, as the other car brands are started manufacturing automobiles before the registration year.

As such, we will remove all entries outside of the 1927 - 2016 range.

In [24]:
autos = autos.loc[(autos['registration_year'] > 1910) & (autos['registration_year'] < 2017), :]

In [25]:
autos['registration_year'].value_counts(normalize=True).head(15)

2000    0.067230
2005    0.062910
1999    0.062028
2004    0.058031
2003    0.057966
2006    0.057386
2001    0.056569
2002    0.053302
1998    0.050551
2007    0.048875
2008    0.047564
2009    0.044770
1997    0.041739
2011    0.034883
2010    0.034152
Name: registration_year, dtype: float64

We see that most of the vehicles are from the past couple of decades, which is to be expected.

In [26]:
autos['brand'].value_counts(normalize=True).head(15)

volkswagen       0.211361
bmw              0.109958
opel             0.107336
mercedes_benz    0.096460
audi             0.086616
ford             0.069981
renault          0.047112
peugeot          0.029875
fiat             0.025684
seat             0.018269
skoda            0.016421
nissan           0.015303
mazda            0.015238
smart            0.014207
citroen          0.014035
Name: brand, dtype: float64

We see that the brands `volkswagen`, `bmw`, `opel`, `mercedez_benz`, `audi`, and `ford` are the top 6 most listed brands, and make up more than 50% of the listings.

In [27]:
top6 = autos['brand'].value_counts().head(6).index

In [28]:
brand_mean_prices = {}

for brand in top6:
    mean = autos.loc[autos['brand'] == brand, 'price'].mean()
    brand_mean_prices[brand] = mean
    
for k, v in brand_mean_prices.items():
    print('{:<20}{}'.format(k, v))

volkswagen          5417.789505796217
bmw                 8367.020719311962
opel                2991.811974369243
mercedes_benz       8657.285427807486
audi                9362.169478908188
ford                3757.5282555282556


In the last cell, we aggregated across brands to understand mean price. We observed that in the top 6 brands, there's a distinct price gap.

* Audi, BMW and Mercedes Benz are more expensive
* Ford and Opel are less expensive
* Volkswagen is in between

For the top 6 brands, let's use aggregation to understand the average odometer readings for those cars to see if there's any visible link with mean price.

In [29]:
brand_mean_odometer = {}

for brand in top6:
    mean = autos.loc[autos['brand'] == brand, 'odometer_km'].mean()
    brand_mean_odometer[brand] = mean
    
for k, v in brand_mean_odometer.items():
    print('{:<20}{}'.format(k, v))

volkswagen          128708.56213138092
bmw                 132553.75293197812
opel                129324.1890268322
mercedes_benz       130838.9037433155
audi                129208.43672456576
ford                124210.68796068797


In [30]:
mean_price = pd.Series(brand_mean_prices).sort_values(ascending=False)
mean_odometer = pd.Series(brand_mean_odometer)
df = pd.DataFrame(mean_price, columns=['mean_price'])
df['mean_odometer_km'] = mean_odometer
df

,mean_price,mean_odometer_km
audi,9362.169479,129208.436725
mercedes_benz,8657.285428,130838.903743
bmw,8367.020719,132553.752932
volkswagen,5417.789506,128708.562131
ford,3757.528256,124210.687961
opel,2991.811974,129324.189027


In [31]:
mean_odometer.std()

2795.0973000575864

We can see that while the prices between the brands vary, the average odometer readings are close, with a standard deviation of about 2800 km. Based on this, we can conclude that the price variations are mostly due to the brand, and less so due to the odometer readings.